# Hands-on: Text Processing
This hands-on will cover the necessary steps in a text processing pipeline for Human Language Technologies (HLT). Some examples of projects and tasks in which this pipeline will be useful are the following:
- **Language Translation** - translation of a sentence or body of text from one language to another
- **Word Sense Disambiguation** - determining the meaning and context of a polysemic word in a body of text
- **Sentiment Analysis** - determining the overall sentiment towards a certain topic or word, whether it's positive, negative, or neutral
- **Topic Modeling** - identifying the different topics discusses in a text and determining the most prevalent one

And there are others more like question answering, information extraction, and more recently, detecting mis/disinformation.

## Pre-processing Pipeline

- **Tokenization** — split sentences into words and symbols
- **Convert to lowercase**
- **Removing unnecessary punctuation, tags, and emojis**
- **Removing stop words** — removing frequently occurring words like articles (e.g. ”the”, ”is”, etc.) that do not have specific meanings
- **Stemming** — transforms a word to their root form by removing inflectional endings. It is done by usually dropping the suffixes.

```
The stemmed form of cries is: cri
The stemmed form of crying is: cry
```

- **Lemmatization** — properly removing inflectional endings by determining the part of speech and doing morphological analysis. It transforms words to their base or dictionary form.

```
The lemmatized form of cries is: cry
The lemmatized form of crying is: cry
```

> **NOTE:** Not all HLT tasks/projects will follow the same pipeline. For example, topic modeling were proven to be better with stop words, so the removal of stop words is typicallly skipped. 

In [9]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim 

from gensim.parsing.preprocessing import STOPWORDS

import nltk

### Load JSON dataset containing posts from r/waze
This dataset was collected in 2019 using the pushshift.io API. It contains the `submission ID` and the post's `body` of text.

In [2]:
json_file = open('C:/Users/Ronell Roxas/Desktop/DATASCI/git_datasets/raw_textonly_waze_2019.json')
json_data = json.load(json_file)
documents = pd.DataFrame(json_data)

In [3]:
documents.head(5)

,body,id
0,Seriously Waze whomever is in charge of UX nee...,7nraji
1,"I agree. And what's with the blue ""Go"" buttons...",ds4cv5q
2,"Yes! When selecting a destination from seach, ...",ds4mc2j
3,I totally agree with you and I will add: make ...,ds4mtd9
4,I agree and disagree with little bits and piec...,ds52syn


### Remove URLs from text
You can use [Regex101](https://regex101.com/) for checking your regular expressions

In [4]:
def removeURLFromText(text):
    result = re.sub(r"http\S+", "", text)
    result.strip()
    return result

In [5]:
processed_docs = documents['body'].map(removeURLFromText)
processed_docs.head()

0    Seriously Waze whomever is in charge of UX nee...
1    I agree. And what's with the blue "Go" buttons...
2    Yes! When selecting a destination from seach, ...
3    I totally agree with you and I will add: make ...
4    I agree and disagree with little bits and piec...
Name: body, dtype: object

### Pre-process post's body
`gensim`'s `simple_preprocess()` converts a text into a list of tokens that are already in lowercase.

This step also removes stop words and words with less than 3 characters.

In [29]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize(token))
    return result

In [30]:
preprocess("I worked my whole life, just to get right")

['work', 'life', 'right']

### Stem and lemmatize per text
Lemmatize the word first. If there will be words missed, the stemmer should be able to handle it. The lemmatization will only be done for verbs.

In [28]:
nltk.download('wordnet')

def lemmatize(text):
    return nltk.stem.WordNetLemmatizer().lemmatize(text, pos='v')

[nltk_data] Downloading package wordnet to C:\Users\Ronell
[nltk_data]     Roxas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
processed_docs = processed_docs.map(preprocess)
processed_docs.head()

0    [seriously, waze, whomever, charge, ux, need, ...
1    [agree, blue, button, stay, screen, second, ye...
2    [yes, select, destination, seach, present, opt...
3    [totally, agree, add, screen, mind, optional, ...
4    [agree, disagree, little, bits, piece, items, ...
Name: body, dtype: object

### Create a `gensim` Dictionary
This will organize your bag of words into word <-> id mappings

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [33]:
for x in range(20):
    print(x, ":", dictionary[x])

0 : accidents
1 : accurate
2 : ads
3 : ago
4 : app
5 : area
6 : aspect
7 : avoid
8 : bad
9 : bar
10 : base
11 : behalf
12 : better
13 : brainer
14 : briefly
15 : button
16 : car
17 : center
18 : change
19 : charge


### Filter words
Filter out tokens that appear in

- less than `no_below` documents (absolute number) or
- more than `no_above` documents (fraction of total corpus size, not absolute number).
- after (1) and (2), keep only the first `keep_n` most frequent tokens (or keep all if `None`).

In [34]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [36]:
dictionary.dfs

{83: 46,
 104: 4471,
 16: 114,
 64: 636,
 8: 202,
 99: 1066,
 15: 500,
 55: 44,
 66: 169,
 107: 282,
 20: 92,
 89: 64,
 17: 340,
 34: 559,
 45: 450,
 69: 79,
 23: 16,
 3: 282,
 102: 749,
 92: 909,
 41: 858,
 91: 790,
 39: 805,
 76: 65,
 52: 22,
 12: 302,
 21: 113,
 85: 273,
 43: 369,
 30: 230,
 105: 62,
 4: 1446,
 79: 23,
 95: 38,
 24: 97,
 51: 23,
 9: 62,
 61: 168,
 48: 63,
 78: 33,
 27: 53,
 10: 186,
 98: 269,
 50: 53,
 36: 27,
 62: 146,
 74: 154,
 31: 142,
 46: 130,
 97: 101,
 80: 44,
 19: 50,
 59: 1709,
 100: 151,
 42: 440,
 25: 457,
 73: 62,
 86: 499,
 57: 169,
 94: 773,
 56: 411,
 47: 74,
 32: 34,
 90: 359,
 22: 30,
 6: 19,
 81: 439,
 53: 283,
 68: 38,
 1: 160,
 54: 1154,
 63: 15,
 58: 134,
 93: 37,
 26: 59,
 38: 18,
 33: 88,
 13: 662,
 14: 33,
 84: 102,
 71: 35,
 2: 83,
 72: 141,
 77: 478,
 49: 142,
 29: 1200,
 60: 336,
 11: 311,
 75: 29,
 82: 200,
 103: 886,
 67: 502,
 40: 28,
 18: 25,
 0: 44,
 87: 80,
 44: 204,
 65: 41,
 7: 233,
 35: 143,
 28: 229,
 37: 87,
 96: 24,
 5: 393,
 

### Map bag of words per document

So far, we have only counted the occurrence of each word across all documents. Next, we need to know how often each word appeared in each document, but now using the IDs generated in the previous step.

In [37]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [38]:
bow_corpus

[[(0, 3),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 3),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 4),
  (10, 2),
  (11, 1),
  (12, 3),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 2),
  (17, 2),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 3),
  (31, 2),
  (32, 2),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 2),
  (41, 3),
  (42, 2),
  (43, 3),
  (44, 1),
  (45, 2),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 3),
  (50, 1),
  (51, 3),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 2),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 2),
  (61, 1),
  (62, 4),
  (63, 1),
  (64, 6),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1),
  (83, 1),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1)

### Compute the TF-IDF per word in a document

- **Term Frequency (TF)** is the number of times token `t` appears in a document divided by the total number of tokens in the document.
- **Inverse Document Frequency (IDF)** is the log(N/n), where, `N` is the number of documents and `n` is the number of documents a token t has appeared in. A less frequently used word will have a high IDF, whereas the IDF of a frequent word is likely to be low. 

We calculate TF-IDF value of a term as = **TF * IDF**

Example:
```
Document 1: "I worked my whole life, just to get right, just to be like"
[work, 1]
[whole, 1]
[life, 1]
[just, 2]
[right, 1]
[like, 1]
```

```
Document 2: "I worked my whole life, just to get high, just to realize"
[work, 1]
[whole, 1]
[life, 1]
[just, 2]
[high, 1]
[realize, 1]
```

```
TF('just',Document1) = 2/7, IDF('just')=log(2/2) = 0
TF('right',Document1) = 1/7,  IDF(‘right’)=log(2/1) = 0.30

TF-IDF(‘just’, Document1) = (2/7)*0 = 0
TF-IDF(‘right’, Document1) = (1/7)*0.30 = 0.42
```

In [42]:
tfidf = gensim.models.TfidfModel(bow_corpus)

In [45]:
corpus_tfidf = tfidf[bow_corpus]
for x in corpus_tfidf:
    print(x)

[(0, 0.2096025716604758), (1, 0.05389201450506542), (2, 0.06201392449027459), (3, 0.046878874782289824), (4, 0.07995186279742096), (5, 0.04277168878170521), (6, 0.08025916513037311), (7, 0.0492408118467775), (8, 0.10201510704301416), (9, 0.2624947765051454), (10, 0.10405744870327632), (11, 0.045667569331981815), (12, 0.13809288818887272), (13, 0.036318841065058845), (14, 0.07342749597163746), (15, 0.0397918858246157), (16, 0.11617344885198949), (17, 0.08912866719415768), (18, 0.07686309841889621), (19, 0.06828562748916601), (20, 0.12147995370432736), (21, 0.05819575310286284), (22, 0.07460692844933799), (23, 0.16477151233923149), (24, 0.06008507934592348), (25, 0.04090467483439381), (26, 0.06623744014265597), (27, 0.06756456870171373), (28, 0.0494550972839042), (29, 0.05791648985065459), (30, 0.14820353114122115), (31, 0.1107377836321259), (32, 0.14611615052325833), (33, 0.06129005295709508), (34, 0.07682319517697114), (35, 0.05528205163671145), (36, 0.07591073056459197), (37, 0.061431

[(85, 0.27606420192220055), (94, 0.4017226598406145), (146, 0.2521697210806849), (150, 0.3169635478419709), (160, 0.20208680848787053), (194, 0.23409571297921805), (278, 0.22287309528699312), (334, 0.23306664047267767), (355, 0.2606325399356716), (460, 0.31649888791499015), (686, 0.2910060018993061), (1067, 0.3820134624943336)]
[(142, 0.4534501611898964), (160, 0.24253534798453796), (231, 0.3792933944759209), (267, 0.2418534331396699), (364, 0.21736069564314317), (550, 0.3983678698745741), (1323, 0.5720667463867201)]
[(113, 1.0)]
[(15, 0.41607636230836276), (142, 0.33830676812268495), (205, 0.3250026752518912), (492, 0.22947128277400858), (686, 0.2605674422549542), (780, 0.3257965101005404), (800, 0.34740561491260036), (910, 0.33904603860658566), (1002, 0.38008360568062916)]
[(54, 0.4786101845664521), (91, 0.5548424259035672), (319, 0.6805014134053735)]
[(94, 0.6230397471571281), (146, 0.7821901772985788)]
[(41, 0.11762773418936588), (69, 0.2224868718763643), (91, 0.1212578276140188), 

[(59, 0.2734880229814236), (104, 0.14108941853055493), (291, 0.29366681417329915), (419, 0.48753675925473017), (1014, 0.7624734670092608)]
[(122, 0.4910754941963574), (160, 0.3098316116340595), (170, 0.45444534779112233), (668, 0.6755210265439933)]
[(29, 0.13443029267037102), (112, 0.4352432872428073), (211, 0.34126216837812945), (278, 0.17719531257150076), (373, 0.2777552309751383), (518, 0.3230488385124614), (728, 0.2662275312142976), (821, 0.3243398109408699), (973, 0.3169963135618023), (1167, 0.262996792486018), (1220, 0.3426336579403007)]
[(29, 0.15038964196647153), (41, 0.17194904410134823), (42, 0.214867589912698), (90, 0.45588520815793593), (102, 0.18068051570683544), (298, 0.4239601051484346), (492, 0.22794260407896796), (782, 0.396654674190312), (1002, 0.37755158640869957), (1062, 0.3599881892300413)]
[(160, 0.3633481106582549), (423, 0.6868526990056621), (661, 0.6294533504158302)]
[(54, 0.2755114715828626), (349, 0.26718811890019256), (768, 0.5530864934740928), (1317, 0.7394

[(4, 0.08447291935363441), (98, 0.1504404062688161), (99, 0.19286324078445044), (102, 0.11027467956179685), (104, 0.04019724466210987), (107, 0.14858923620981224), (165, 0.11418099303570386), (193, 0.09369606849932013), (209, 0.1342944633533623), (210, 0.16533645378221273), (214, 0.08950176258339368), (291, 0.16733496957335614), (300, 0.17690555125471005), (311, 0.18276144265447827), (358, 0.18696796467230456), (365, 0.20213847136174232), (367, 0.2657429107036961), (384, 0.18925346267086965), (473, 0.19899893341470593), (483, 0.15300099222654398), (593, 0.23273885292456978), (632, 0.2988655604592583), (635, 0.3359634447391021), (761, 0.16798172236955106), (913, 0.22422486994028162), (948, 0.22618644292681503), (973, 0.21644097218297875), (1183, 0.22618644292681503)]
[(115, 0.5323137224505834), (241, 0.5188989981523464), (317, 0.6688691431119352)]
[(1283, 1.0)]
[(568, 0.6090324939029091), (744, 0.7931452712904509)]
[(34, 0.2884861116484281), (91, 0.25634017061857534), (489, 0.3730648386

[(4, 0.2562056185852908), (190, 0.39323689851466853), (193, 0.2841793485126576), (313, 0.40470559574329945), (326, 0.5249779193182236), (635, 0.5094870808304995)]
[(29, 0.03822399646957282), (39, 0.0447451702922271), (48, 0.08635991391033261), (49, 0.07308524155240839), (64, 0.048594219898662264), (74, 0.07176012134899849), (77, 0.053259066284357975), (94, 0.454077378492648), (99, 0.040158099218314654), (103, 0.04317913499805913), (104, 0.01673978859712312), (111, 0.1542166032525372), (115, 0.06646229673552517), (126, 0.08688686901927715), (153, 0.43313995985370685), (189, 0.10658654805647699), (211, 0.19406941188873011), (224, 0.07338389852385374), (274, 0.07653003976210103), (279, 0.24987370055490238), (291, 0.03484258733540003), (347, 0.13417521062186982), (364, 0.040942793406741805), (367, 0.05533314762935978), (407, 0.061309485879564606), (416, 0.046409882508172184), (456, 0.09046493196714399), (488, 0.0691214236747276), (584, 0.05713624060790733), (592, 0.09114562178018226), (638

[(41, 0.23012547078039544), (103, 0.11368173666879136), (130, 0.13022987704302225), (186, 0.0894910804749342), (193, 0.10272870506218712), (265, 0.3795511680920076), (267, 0.11994039447407343), (291, 0.2751999898909753), (354, 0.20581150512556307), (425, 0.19894668249874914), (438, 0.23645505397308195), (669, 0.25927448329084546), (739, 0.20664657117569404), (798, 0.22669055041437902), (847, 0.18198242009052631), (872, 0.2622414907036143), (894, 0.21873804840550043), (1258, 0.2448046460637507), (1376, 0.27891717298967256), (1412, 0.27891717298967256)]
[(104, 0.09440358949509031), (140, 0.45214425395521696), (142, 0.2408432939127193), (337, 0.543839340192253), (523, 0.43737409242502456), (706, 0.4915187880616927)]
[(113, 0.861124654249524), (291, 0.5083938727439953)]
[(4, 0.1352181252868223), (34, 0.194890176418293), (41, 0.16798931366701972), (81, 0.2100623666585656), (104, 0.06434483508438572), (112, 0.2378508770838068), (121, 0.24566483090894567), (137, 0.5341876402553604), (186, 0.1

[(7, 0.2517437852855494), (104, 0.1296730311932947), (142, 0.16541150676195493), (165, 0.1841692831941363), (186, 0.1316535845314701), (195, 0.22013605286378082), (199, 0.3015716987432195), (214, 0.1443626913845554), (261, 0.15207596092839132), (267, 0.529346034671683), (398, 0.24148909285052808), (921, 0.3170463586216368), (986, 0.28397100975127754), (1002, 0.3716756969423094)]
[]
[(102, 0.21011809805625026), (163, 0.29643159308104183), (192, 0.22681997207024857), (214, 0.17053724573433088), (272, 0.34157377117554155), (390, 0.38620124980401677), (508, 0.23238732634102055), (538, 0.4094768375000569), (611, 0.3213885183196774), (1380, 0.44346213758798)]
[(35, 0.25832654979018776), (128, 0.32391231142128696), (224, 0.1298948163608788), (364, 0.14494342050092648), (374, 0.20181224178269042), (390, 0.29881386038832625), (439, 0.243124132585882), (782, 0.3569042990767087), (928, 0.329172791362542), (1077, 0.3157213128679405), (1178, 0.37816813350871387), (1386, 0.3505898057655849)]
[(35, 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

